In [1]:
# importar el data set
from google.colab import files
uploaded = files.upload()

Saving cardio_train_normalizado.csv to cardio_train_normalizado.csv


In [15]:

# abrir el dataset
import pandas as pd

df = pd.read_csv("cardio_train_normalizado.csv")


# separar los datos entre los que si tienen ataques y los que no
sin_ataque = df[(df["cardio"] == 0)]
con_ataque = df[(df["cardio"] == 1)]



# eliminar 42 registros del df sin ataque para tener ambos conjuntos iguales
filas_eliminadas = sin_ataque.sample(n=42)
sin_ataque = sin_ataque.drop(filas_eliminadas.index)

# separar los datos entre deentrenamiento y de prueba
# Esta cantidad equivale aproximadamente a una proporción de 70-30
datos_train =sin_ataque.sample(n=24485)
datos_test = sin_ataque.drop(datos_train.index)


# extraer los arreglos de cada dataframe
datos_train = datos_train.values
datos_test = datos_test.values

# extraer los targests
y_train = datos_train[:,-1]
y_test = datos_test[:,-1]

# extraer la data con las variables
x_train = datos_train[:, :-2]
x_test = datos_test[:, :-2]


# aplicar el mismo proceso para el conjunto con ataque

# primer igualar la cantidad de x_test junto con la de con ataques
cantidad = len(con_ataque) - len(x_test)
filas_eliminar = con_ataque.sample(n=cantidad)
con_ataque = con_ataque.drop(filas_eliminar.index)
con_ataque = con_ataque[["age", "gender", "height", "weight", "ap_hi", "ap_lo", "cholesterol", "gluc", "smoke", "alco", "active"]]

con_ataque = con_ataque.values
print(len(con_ataque))

10494


In [18]:
# scalar los datos
from sklearn.preprocessing import MinMaxScaler
import numpy as np

min_max_scaler = MinMaxScaler()
x_train_s = min_max_scaler.fit_transform(x_train)
x_test_s = min_max_scaler.transform(x_test)

# aplicar esto mismos para los datos con ataque

con_ataque_s = min_max_scaler.transform(con_ataque)
con_ataque_s = np.split(con_ataque, 2)
print("done")

done


In [21]:
# crear el auto encoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
np.random.seed(23)



In [22]:
# Como tenemos 25 000 datos de entrenamiento vamos a dividirlos en 5 grupos

sub_x_train_s = np.split(x_train_s, 5)

for subarray in sub_x_train_s:
  dim_entrada = x_train_s.shape[1]
  entrada = Input(shape=(dim_entrada,))
  # Encoder =
  encoder = Dense(11, activation="relu")(entrada)
  encoder = Dense(9, activation="relu")(encoder)
  encoder = Dense(7, activation="relu")(encoder)
  encoder = Dense(5, activation="relu")(encoder)
  encoder = Dense(3, activation="relu")(encoder)

  # Decoder
  decoder = Dense(3, activation="relu")(encoder)
  decoder = Dense(5, activation="relu")(decoder)
  decoder = Dense(7, activation="relu")(decoder)
  decoder = Dense(9, activation="relu")(decoder)
  decoder = Dense(11, activation="sigmoid")(decoder)



autoencoder = Model(inputs=entrada, outputs=decoder)
autoencoder.summary()
print("entrenado")




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense_40 (Dense)            (None, 11)                132       
                                                                 
 dense_41 (Dense)            (None, 9)                 108       
                                                                 
 dense_42 (Dense)            (None, 7)                 70        
                                                                 
 dense_43 (Dense)            (None, 5)                 40        
                                                                 
 dense_44 (Dense)            (None, 3)                 18        
                                                                 
 dense_45 (Dense)            (None, 3)                 12    

In [41]:
 # optimización y entrenamiento
# la perdida será el error absoluto medio

autoencoder.compile(optimizer="adam", loss="mae")
for i in sub_x_train_s:
  historia = autoencoder.fit(i, i,
                             epochs=25,
                             batch_size=512,
                             validation_data=(i, i),
                             shuffle=True)


Epoch 1/25
10/10 [==============================] - 2s 30ms/step - loss: 0.0781 - val_loss: 0.0765
Epoch 2/25
10/10 [==============================] - 0s 8ms/step - loss: 0.0766 - val_loss: 0.0762
Epoch 3/25
10/10 [==============================] - 0s 9ms/step - loss: 0.0762 - val_loss: 0.0761
Epoch 4/25
10/10 [==============================] - 0s 7ms/step - loss: 0.0761 - val_loss: 0.0761
Epoch 5/25
10/10 [==============================] - 0s 8ms/step - loss: 0.0761 - val_loss: 0.0760
Epoch 6/25
10/10 [==============================] - 0s 8ms/step - loss: 0.0760 - val_loss: 0.0760
Epoch 7/25
10/10 [==============================] - 0s 8ms/step - loss: 0.0760 - val_loss: 0.0761
Epoch 8/25
10/10 [==============================] - 0s 10ms/step - loss: 0.0760 - val_loss: 0.0760
Epoch 9/25
10/10 [==============================] - 0s 16ms/step - loss: 0.0760 - val_loss: 0.0760
Epoch 10/25
10/10 [==============================] - 0s 9ms/step - loss: 0.0760 - val_loss: 0.0760
Epoch 11/25
10/1

In [26]:
# reconstruir el conjunto
import tensorflow as tf

# dividir el xtest en  grupos
sub_x_test = np.split(x_test_s, 2)

# reconstruir los datos de prueba para los sin ataque
rec_1 = autoencoder.predict(sub_x_test[0])
rec_2 = autoencoder.predict(sub_x_test[1])
# sacar el error medio de las reconstrucciones anteriores
loss_1 = tf.keras.losses.mae(rec_1, sub_x_test[0])
loss_2 = tf.keras.losses.mae(rec_2, sub_x_test[1])

# reconstruir los datos con ataques
rec_con_ataque_1 = autoencoder.predict(con_ataque_s[0])
rec_con_ataque_2 = autoencoder.predict(con_ataque_s[1])
# extraer errores para estos
loss_con_ataque_1 = tf.keras.losses.mae(rec_con_ataque_1, con_ataque_s[0])
loss_con_ataque_2 = tf.keras.losses.mae(rec_con_ataque_2, con_ataque_s[1])






164/164 [==============================] - 0s 2ms/step


In [27]:
# Calcular predicciones individuales
def predecir(modelo, datos, umbral):
  reconstrucciones = modelo(datos)
  perdida = tf.keras.losses.mae(reconstrucciones, datos)
  return tf.math.less(perdida, umbral)

def calcular_sensitividad(prediccion, titulo):
  TP = np.count_nonzero(~prediccion)
  FN = np.count_nonzero(prediccion)
  sen = 100*(TP/(TP+FN))

  print(titulo + ': {:.1f}%'.format(sen))

def calcular_especificidad(prediccion, titulo):
  TN = np.count_nonzero(prediccion)
  FP = np.count_nonzero(~prediccion)
  esp = 100*(TN/(TN+FP))

  print(titulo + ': {:.1f}%'.format(esp))
  return esp

In [39]:
## predicir

umbral = np.mean(loss_1 + np.std(loss_1))
# precisión para los datos sin ataques
pred_1 = predecir(autoencoder, sub_x_test[0], umbral)
esp = calcular_especificidad(pred_1, "porcentaje para los sin ataques grupo 1")

pred_2 = predecir(autoencoder, sub_x_test[1], umbral)
esp = calcular_especificidad(pred_2, "porcentaje para los sin ataques grupo 2")


# predicción para los datos con ataques
pred_3 = predecir(autoencoder, con_ataque_s[0], umbral)
sen_1 = calcular_sensitividad(pred_3, "porcentaje para los con ataques grupo 1")
pred_4 = predecir(autoencoder, con_ataque_s[1], umbral)
sen_1 = calcular_sensitividad(pred_4, "porcentaje para los con ataques grupo 2")



porcentaje para los sin ataques grupo 1: 84.9%
porcentaje para los sin ataques grupo 2: 85.0%
porcentaje para los con ataques grupo 1: 100.0%
porcentaje para los con ataques grupo 2: 100.0%


In [45]:
# guardar el modelo

autoencoder.save("autoencoder.h5")
print("guardado")

# Descargar el archivo
from google.colab import files
files.download("autoencoder.h5")
print("descargado")


guardado


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

descargado
